2.5.1+cu124


In [21]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,EncoderDecoderCache
# the cool_fusion function are in source/cool_fusion.py
from source.cool_fusion import CoolFusion
from lm_eval.api.model import LM
from lm_eval import evaluator

In [11]:
class CoolFusionWrapper(LM):
    def __init__(self, fused_model):
        super().__init__()
        self.fused_model = fused_model
    
    @property
    def max_length(self):
        return self.fused_model.max_length
    
    def generate_until(self, requests):
        generations = []
        for context, stop in requests:
            gen = self.fused_model.generate(context)
            for s in stop:
                if s in gen:
                    gen = gen.split(s)[0]
            generations.append(gen)
        return generations
    def loglikelihood(self, requests):
        raise NotImplementedError("This model does not support loglikelihood.")

    def loglikelihood_rolling(self, requests):
        raise NotImplementedError("This model does not support loglikelihood_rolling.")
    

In [5]:
from huggingface_hub import login
login("your key here")

HTTPError: Invalid user token. If you didn't pass a user token, make sure you are properly logged in by executing `huggingface-cli login`, and if you did pass a user token, double-check it's correct.

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define the model names
model_name1 = "deepseek-ai/deepseek-coder-1.3b-instruct"
model_name2 = "Salesforce/codegen2-1B"

# Load tokenizers
tokenizer1 = AutoTokenizer.from_pretrained(model_name1)
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)

# Load models (using CPU by default; add device_map="auto" for GPU)
model1 = AutoModelForCausalLM.from_pretrained(model_name1)
model2 = AutoModelForCausalLM.from_pretrained(model_name2)

# Sample prompt for generation
prompt = "Once upon a time"

# # Prepare inputs
# input_ids1 = tokenizer1(prompt, return_tensors="pt").input_ids
# input_ids2 = tokenizer2(prompt, return_tensors="pt").input_ids

# # Generate text with both models
# output1 = model1.generate(input_ids1, max_new_tokens=50, do_sample=True)
# output2 = model2.generate(input_ids2, max_new_tokens=50, do_sample=True)

# # Decode and print outputs
# print("GPT-2 Output:")
# print(tokenizer1.decode(output1[0], skip_special_tokens=True))
# print("\nGPT-Neo Output:")
# print(tokenizer2.decode(output2[0], skip_special_tokens=True))


tokenizer_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

C:\Users\atous\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\atous\.cache\huggingface\hub\models--Salesforce--codegen2-1B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen2-1B were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CodeGenForCausalLM

In [19]:
# usage
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"
fused_model = CoolFusion(
    models={"deepseek-ai/deepseek-coder-1.3b-instruct": model1, "Salesforce/codegen2-1B": model2},
    tokenizers={"deepseek-ai/deepseek-coder-1.3b-instruct": tokenizer1, "Salesforce/codegen2-1B": tokenizer2},
    max_length=250
)
wrapper = CoolFusionWrapper(fused_model)

results = evaluator.simple_evaluate(

    model = wrapper,
    
    tasks = ["mbpp"],
    num_fewshot=3,
    
    
)


NotImplementedError: This metric is currently not supported on Windows.